PCSEA High Level Interface
==
This notebook demonstrates the use of the high level interface to PCSEA.  This interface permits analysis of more complex systems, as well as identification of possible phase separation behavior.

In [1]:
# Import necessary modules
import inspect, io, math, numpy, os, sys
sys.path.append(os.path.realpath(os.path.join(os.path.dirname(inspect.getfile(inspect.currentframe())), "..")))
import crystal, potential, equilibrium

In [2]:
# Create cells for testing
NaCl = crystal.Cell(
    numpy.array([[2, 0, 0], [0, 2, 0], [0, 0, 2]]), [
    numpy.array([[0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 0]]),
    numpy.array([[0, 0, 1], [0, 1, 0], [1, 0, 0], [1, 1, 1]])])
CsCl = crystal.Cell(
    numpy.array([[2, 0, 0], [0, 2, 0], [0, 0, 2]]), [
    numpy.array([[0, 0, 0]]),
    numpy.array([[1, 1, 1]])])

Scaling
==
It is desirable to rescale cells and potentials such that interacting neighboring atoms rest at equilibrium positions, or positions that are at least very close to equilibrium.  Cells can be rescaled in place very easily.  Minima of potentials can be measured, although this action can be performed automatically during energy analysis.

In [3]:
# Rescale cells
equilibrium.rescale([NaCl, CsCl])

In [4]:
# Note that cells must be remeasured after rescaling
NaCl.measure(10)
CsCl.measure(10)

In [5]:
# Minimum interatomic contacts are now set to 1
([NaCl.contact(i, j) for i in range(NaCl.atom_types()) for j in range(NaCl.atom_types())],
 [CsCl.contact(i, j) for i in range(CsCl.atom_types()) for j in range(CsCl.atom_types())])

([1.4142135623730951, 1.0, 1.0, 1.4142135623730951],
 [1.1547005383792517, 1.0, 1.0, 1.1547005383792517])

In [6]:
# Cell vectors have been changed to accomplish this, although the shape of the cell remains fixed
([NaCl.vector(i) for i in range(NaCl.dimensions())], [CsCl.vector(i) for i in range(CsCl.dimensions())])

([array([ 2.,  0.,  0.]), array([ 0.,  2.,  0.]), array([ 0.,  0.,  2.])],
 [array([ 1.15470054,  0.        ,  0.        ]),
  array([ 0.        ,  1.15470054,  0.        ]),
  array([ 0.        ,  0.        ,  1.15470054])])

In [7]:
# Minima of potentials can be found with a single call
# Dictionary keys can be any sort of hashable object, as is illustrated here with integer keys
# Typically these keys will be tuples for compatibility with other function interfaces
# Usually it will not be necessary to use this functionality directly
lambda_ = numpy.linspace(0.1, 1.0, 10)
lambda_, equilibrium.minima({1: potential.stsp}, [{1: (1.0, 1.0, lambda_i, 96.0, 1.0)} for lambda_i in lambda_],
    [{1: (0.0, 2.0)} for lambda_i in lambda_])

(array([ 0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ]),
 [{1: 1.0244511294825793},
  {1: 1.0170288157921576},
  {1: 1.0127118030941682},
  {1: 1.0096598868361948},
  {1: 1.0072989256743716},
  {1: 1.0053739507339252},
  {1: 1.003749240570426},
  {1: 1.0023439785284511},
  {1: 1.0011060605211579},
  {1: 1.0000000019604902}])

Energy calculation
--
There are a few different calls that can be made to perform batch energy calculations.  The simplest takes a number of cells, potentials, and parameter sets, and performs the energy calculations.

In [8]:
# In this case, no special rescaling of the potential is performed
equilibrium.energies([1, 1], [NaCl, CsCl], {(0, 0): potential.stsp, (1, 1): potential.stsp, (0, 1): potential.stsp},
    [{(0, 0): (1.0, 1.0, 0.5, n, 1.0), (1, 1): (1.0, 1.0, 0.5, n, 1.0), (0, 1): (1.0, 1.0, 1.0, n, 1.0)} for n in \
    numpy.linspace(6.0, 36.0, 6)])

[[-4.8334224986714736,
  -3.1306728769367234,
  -3.0148621237872955,
  -3.0018108691892946,
  -3.0002246936375672,
  -3.0000280291428534],
 [-6.4198187921210366,
  -4.5085487487747358,
  -4.2279439523484976,
  -4.1006067762944332,
  -4.0433104764085437,
  -4.0184302309055768]]

In [9]:
# Here, the potentials are rescaled such that their minima are at r = 1
# It is necessary to specify bounds within which the minima should be searched for
# It may be wise to run equilibrium.minima to make sure that the minima have actually been found properly
equilibrium.energies([1, 1], [NaCl, CsCl], {(0, 0): potential.stsp, (1, 1): potential.stsp, (0, 1): potential.stsp},
    [{(0, 0): (1.0, 1.0, 0.5, n, 1.0), (1, 1): (1.0, 1.0, 0.5, n, 1.0), (0, 1): (1.0, 1.0, 1.0, n, 1.0)} for n in \
    numpy.linspace(6.0, 36.0, 6)], 1.0,
    [{(0, 0): (0.5, 2.0), (1, 1): (0.5, 2.0), (0, 1): (0.5, 2.0)} for n in numpy.linspace(6.0, 36.0, 6)])
# Notice that the energies have decreased as the minima are more well aligned

[[-4.3545173427510688,
  -3.0737345810549517,
  -3.0076659272448913,
  -3.0009105760152943,
  -3.0001122069019885,
  -3.0000139745440895],
 [-5.8658832293392731,
  -4.2885816054771064,
  -4.1187909335061637,
  -4.0510787123333216,
  -4.0217707657660133,
  -4.009227584932602]]

High Level Interface Module Reference
--

In [10]:
help(equilibrium)

Help on module equilibrium:

NAME
    equilibrium - Performs calculations to determine energies of a large number of crystal cells with a large number of potential parameters.

FUNCTIONS
    energies(stoichiometry, cells, potentials, parameter_sets, scale_target=None, search_bounds=None)
        Calculates system energies assuming the formation of a single ordered phase.
        
        Parameters
        ----------
        stoichiometry : list of float
            System solution stoichiometry
        cells : list of :py:class:`crystal.Cell`
            Structures to analyze
        potentials : dict of (tuple of int) -> potential
            Potentials for each pair of atom types
        parameter_sets : list of dict of (tuple of int) -> tuple
            Sets of parameters fed to the potentials
        scale_target : float
            Optional potential minimum distance target
        search_bounds : list of dict of (tuple of int) -> (tuple of float)
            Bounds in which to 